In [1]:
import pandas as pd
from sqlalchemy import create_engine

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [2]:
# Merging WTI, KWH, Houston Weather

In [3]:
Weather_file = "Resources/Houston_Weather_NOOA_NCDC.csv"
Weather_df = pd.read_csv(Weather_file)
Weather_df.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES,...,WT03,WT03_ATTRIBUTES,WT04,WT04_ATTRIBUTES,WT05,WT05_ATTRIBUTES,WT06,WT06_ATTRIBUTES,WT11,WT11_ATTRIBUTES
0,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/1/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/2/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/3/1990,0.03,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/4/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",29.4718,-95.0832,5.8,12/5/1990,0.00,",,0,0800",0.0,"P,,0",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
print(Weather_df.columns)

Index(['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'PRCP',
       'PRCP_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES',
       'TMAX', 'TMAX_ATTRIBUTES', 'TMIN', 'TMIN_ATTRIBUTES', 'TOBS',
       'TOBS_ATTRIBUTES', 'WT01', 'WT01_ATTRIBUTES', 'WT03', 'WT03_ATTRIBUTES',
       'WT04', 'WT04_ATTRIBUTES', 'WT05', 'WT05_ATTRIBUTES', 'WT06',
       'WT06_ATTRIBUTES', 'WT11', 'WT11_ATTRIBUTES'],
      dtype='object')


In [5]:
Weather_df['DATE'] = pd.to_datetime(Weather_df['DATE'])

In [6]:
new_weather_data_df = Weather_df[['DATE','STATION','NAME','TMAX','TMIN','TOBS']]
new_weather_data_df.head()

,DATE,STATION,NAME,TMAX,TMIN,TOBS
0,1990-12-01,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",60.0,44.0,55.0
1,1990-12-02,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",73.0,55.0,60.0
2,1990-12-03,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",76.0,59.0,59.0
3,1990-12-04,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",61.0,36.0,36.0
4,1990-12-05,USC00414333,"HOUSTON NATIONAL WEATHER SERVICE OFFICE, TX US",56.0,28.0,30.0


In [7]:
new_weather_data_df.reset_index(drop=True, inplace=True)

In [8]:
WTI_file = "Resources/WTI_Prices.csv"
WTI_df = pd.read_csv(WTI_file)
WTI_df.head()

,DATE,DCOILWTICO
0,1/14/1986,24.97
1,1/15/1986,25.18
2,1/16/1986,23.98
3,1/17/1986,23.63
4,1/20/1986,21.33


In [9]:
WTI_df['DATE'] = pd.to_datetime(WTI_df['DATE'])
WTI_df.head()

,DATE,DCOILWTICO
0,1986-01-14,24.97
1,1986-01-15,25.18
2,1986-01-16,23.98
3,1986-01-17,23.63
4,1986-01-20,21.33


In [10]:
new_WTI_data = WTI_df[~(WTI_df['DATE'] < '1990-01-01')]
print(new_WTI_data)

           DATE DCOILWTICO
1034 1990-01-01          .
1035 1990-01-02      22.88
1036 1990-01-03      23.81
1037 1990-01-04      23.41
1038 1990-01-05      23.07
...         ...        ...
9255 2021-07-06      73.62
9256 2021-07-07      72.22
9257 2021-07-08      72.98
9258 2021-07-09      74.56
9259 2021-07-12      74.21

[8226 rows x 2 columns]


In [11]:
new_WTI_data_df = pd.DataFrame(new_WTI_data)
new_WTI_data_df

,DATE,DCOILWTICO
1034,1990-01-01,.
1035,1990-01-02,22.88
1036,1990-01-03,23.81
1037,1990-01-04,23.41
1038,1990-01-05,23.07
...,...,...
9255,2021-07-06,73.62
9256,2021-07-07,72.22
9257,2021-07-08,72.98
9258,2021-07-09,74.56


In [12]:
new_WTI_data_df.rename(columns = {'DCOILWTICO':'Daily Crude Oil Price'},inplace = True)

In [13]:
new_WTI_data_df

,DATE,Daily Crude Oil Price
1034,1990-01-01,.
1035,1990-01-02,22.88
1036,1990-01-03,23.81
1037,1990-01-04,23.41
1038,1990-01-05,23.07
...,...,...
9255,2021-07-06,73.62
9256,2021-07-07,72.22
9257,2021-07-08,72.98
9258,2021-07-09,74.56


In [14]:
new_WTI_data_df.reset_index(drop=True, inplace=True)

In [15]:
new_WTI_data_df

,DATE,Daily Crude Oil Price
0,1990-01-01,.
1,1990-01-02,22.88
2,1990-01-03,23.81
3,1990-01-04,23.41
4,1990-01-05,23.07
...,...,...
8221,2021-07-06,73.62
8222,2021-07-07,72.22
8223,2021-07-08,72.98
8224,2021-07-09,74.56


In [16]:
KWH_file = "Resources/KWH_Houston.csv"
KWH_df = pd.read_csv(KWH_file)
KWH_df.head()

,DATE,APUS37B72610
0,11/1/1978,0.032
1,12/1/1978,0.034
2,1/1/1979,0.035
3,2/1/1979,0.036
4,3/1/1979,0.038


In [17]:
KWH_df.rename(columns = {'APUS37B72610':'$/KWH'},inplace = True)

In [18]:
KWH_df['DATE'] = pd.to_datetime(KWH_df['DATE'])
WTI_df.head()

,DATE,DCOILWTICO
0,1986-01-14,24.97
1,1986-01-15,25.18
2,1986-01-16,23.98
3,1986-01-17,23.63
4,1986-01-20,21.33


In [19]:
new_KWH_data = KWH_df[~(KWH_df['DATE'] < '1990-01-01')]
print(new_KWH_data)

          DATE  $/KWH
134 1990-01-01  0.072
135 1990-02-01  0.073
136 1990-03-01  0.073
137 1990-04-01  0.073
138 1990-05-01  0.085
..         ...    ...
507 2021-02-01  0.143
508 2021-03-01  0.144
509 2021-04-01  0.141
510 2021-05-01  0.138
511 2021-06-01  0.138

[378 rows x 2 columns]


In [22]:
new_KWH_df = pd.DataFrame(new_KWH_data) 
new_KWH_df.head()

,DATE,$/KWH
134,1990-01-01,0.072
135,1990-02-01,0.073
136,1990-03-01,0.073
137,1990-04-01,0.073
138,1990-05-01,0.085


In [23]:
rds_connection_string = "postgres:postgres@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [24]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine,reflect=True)
Base

sqlalchemy.ext.automap.Base

In [25]:
engine.table_names()

['weather', 'wti', 'kwh']

In [26]:
new_KWH_df.to_sql(name='kwh', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "DATE" of relation "kwh" does not exist
LINE 1: INSERT INTO kwh ("DATE", "$/KWH") VALUES ('1990-01-01T00:00:...
                         ^

[SQL: INSERT INTO kwh ("DATE", "$/KWH") VALUES (%(DATE)s, %($/KWH)s)]
[parameters: ({'DATE': datetime.datetime(1990, 1, 1, 0, 0), '$/KWH': '0.072'}, {'DATE': datetime.datetime(1990, 2, 1, 0, 0), '$/KWH': '0.073'}, {'DATE': datetime.datetime(1990, 3, 1, 0, 0), '$/KWH': '0.073'}, {'DATE': datetime.datetime(1990, 4, 1, 0, 0), '$/KWH': '0.073'}, {'DATE': datetime.datetime(1990, 5, 1, 0, 0), '$/KWH': '0.085'}, {'DATE': datetime.datetime(1990, 6, 1, 0, 0), '$/KWH': '0.085'}, {'DATE': datetime.datetime(1990, 7, 1, 0, 0), '$/KWH': '0.085'}, {'DATE': datetime.datetime(1990, 8, 1, 0, 0), '$/KWH': '0.085'}  ... displaying 10 of 378 total bound parameter sets ...  {'DATE': datetime.datetime(2021, 5, 1, 0, 0), '$/KWH': '0.138'}, {'DATE': datetime.datetime(2021, 6, 1, 0, 0), '$/KWH': '0.138'})]
(Backgrou